# Agentic workflow with Agent Engine A2A integration example

This notebook shows an example of an agentic workflow using remote A2A agents deployed on Agent Engine.

See the following notebook for the workflow description.

- [Multi-agent workflow example](https://github.com/google-cloud-japan/sa-ml-workshop/blob/main/blog/multi_agent_article_writing_example_en.ipynb)

## Preparation

In [ ]:
%pip install --user \
    google-adk==1.14.1 \
    google-genai==1.36.0 \
    google-cloud-aiplatform==1.113.0 \
    a2a-sdk==0.3.5

In [13]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

In [ ]:
import httpx, json, os, uuid
from google.auth import default
from google.auth.transport.requests import Request
import vertexai

# ADK
from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.auth.credential_service.in_memory_credential_service import InMemoryCredentialService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, VertexAiSessionService

# A2A
from a2a.client import ClientConfig, ClientFactory
from a2a.types import AgentCard, TransportProtocol
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.a2a.utils.agent_card_builder import AgentCardBuilder

# Agent Engine
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card


[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

Local application class to execute the root agent.

In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## Define agents to be deployed as remote A2A agents

In [3]:
instruction = '''
Your role is to gather the necessary information for writing an article and compile it into a research report.
You will create a list of about 5 topics to be used as a reference when writing an article on a specified theme.
A subsequent agent will create the research report based on this list.

* Output Format
Output in English.
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='''
An agent that gathers the necessary information for writing an article
and compiles it into a report (theme selection)
    ''',
    instruction=instruction,
)

In [4]:
instruction = '''
Your role is to collect the necessary information to write an article and compile it into a research report.
The preceding agent will specify about five research topics.

* Output format
Output in Enlgish.
The research report should summarize objective information for each topic.
Each topic must be described in five sentences or more.
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='''
An agent that collects the information necessary for writing an article
and compiles it into a report (report creation)
    ''',
    instruction=instruction,
)

In [5]:
instruction = '''
Your role is to write a light, feature article on a specific topic.
You will be given the article's "theme" and a "research report" related to its content.
Based on the objective facts described in the research report, please write a reliable feature article.

If a reviewer points out corrections, please rewrite the article you just wrote according to the feedback.

* Output Conditions
- Output in Enlgish.
- The content should be something that can be casually read in a few minutes, assuming the reader has some basic knowledge of the topic.
- Use a relatively casual and conversational tone.
- Do not include an article title; use the following structure. Arrange each section title to fit the content.
  0. Introduction: Without a section title, write a one- or two-sentence introduction that will make someone want to read the article.
  1. Overview: Summarize and briefly explain the overall picture of the topic.
  2. Latest Information: Feature new information that is particularly noteworthy.
  3. Practical Application: Introduce one thing related to the topic that readers might want to try for themselves.
  4. Conclusion

- Use markdown ## headers for each section title. Use markdown for bullet points as needed.
- Do not use any other markdown formatting.
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='An agent that writes feature articles on specific topics.',
    instruction=instruction,
)

In [6]:
instruction = f'''
Your role is to review a feature article and provide comments for improvement to ensure its content meets the article's requirements.

* Article Requirements
- The article must begin with a title of less than about 10 words.
  The title should contain useful life information that is applicable from today, making the reader feel they "must read it now."
  The title must use a markdown # header.
- Immediately after the title, add an introduction that summarizes "why this topic is being addressed now" to motivate the reader.
- Use emojis in each section's subtitle to create a friendly feel.
- It must introduce at least three concrete examples that the reader can put into practice starting today.

* Output Format
- Output in English.
- First, explain the good points of the article.
- Next, list the points for correction in a bulleted format.
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='An agent that reviews feature articles.',
    instruction=instruction,
)

## Deploy agents on Agent Engine

In [7]:
# Wrapper class of VertexAiSessionService that's compatible with InMemorySessionService
class MyVertexAiSessionService(VertexAiSessionService):

    async def create_session(self, app_name, user_id, state={}, session_id=None):
        # Ignore session_id
        session = await super().create_session(
            app_name=app_name,
            user_id=user_id,
            state=state,
        )
        return session

    async def get_session(self, app_name, user_id, session_id):
        # Return None if session non exists.
        try:
            session = await super().get_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id,
            )
            return session
        except:
            return None


def get_create_runner_class(agent, resource_name):
    async def create_runner():
        return Runner(
            # Use resource_name as app_name for VertexAiSessionService.
            app_name=resource_name or agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=MyVertexAiSessionService(),
            memory_service=InMemoryMemoryService(),
            credential_service=InMemoryCredentialService(),
        )
    return create_runner


def get_agent_executor_class(agent, resource_name):
    def agent_executor_builder():
        return A2aAgentExecutor(
            runner=get_create_runner_class(agent, resource_name),
        )       
    return agent_executor_builder


async def get_agent_card(agent):
    builder = AgentCardBuilder(agent=agent)
    adk_agent_card = await builder.build()
    return create_agent_card(
        agent_name=adk_agent_card.name,
        description=adk_agent_card.description,
        skills=adk_agent_card.skills
    )

In [8]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None

In [11]:
agents = {
    'research_agent1_a2a': research_agent1,
    'research_agent2_a2a': research_agent2,
    'writer_agent_a2a': writer_agent,
    'review_agent_a2a': review_agent,
}

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

for agent_name, agent in agents.items():    
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, None)
    )
    config={
        'display_name': agent_name,
        'description': a2a_agent.agent_card.description,
        'requirements': [
            'google-adk==1.14.1',
            'google-genai==1.36.0',
            'google-cloud-aiplatform==1.113.0',
            'a2a-sdk==0.3.5'
        ],
        'http_options': {
            'base_url': f'https://{LOCATION}-aiplatform.googleapis.com',
            'api_version': 'v1beta1',
        },
        'staging_bucket': f'gs://{PROJECT_ID}',
    }

    resource_name = get_agent_resource(agent_name)
    if not resource_name:
        # Deploy dummy agent to assign Agent Engine resource name
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    
    # Redeploy with Agent Engine resource name for VertexAiSessionService
    resource_name = get_agent_resource(agent_name)
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, resource_name)
    )
    remote_a2a_agent = client.agent_engines.update(
        name=resource_name,
        agent=a2a_agent,
        config=config,
    )

/var/tmp/ipykernel_8184/1771281126.py:49: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)


## Define RemoteA2aAgent instances and Root agent

In [16]:
credentials, _ = default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
credentials.refresh(Request())
factory = ClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
        httpx_client=httpx.AsyncClient(
            timeout=60, # This is important.
            headers={
                'Authorization': f'Bearer {credentials.token}',
                'Content-Type': 'application/json',
            }
        ),
    )
)

resource_name = get_agent_resource('research_agent1_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent1_remoteA2a = RemoteA2aAgent(
    name='research_agent1',
    description='''
An agent that gathers the necessary information for writing an article
and compiles it into a report (theme selection)
    ''',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent2_remoteA2a = RemoteA2aAgent(
    name='research_agent2',
    description='''
An agent that collects the information necessary for writing an article
and compiles it into a report (report creation)
    ''',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
writer_agent_remoteA2a = RemoteA2aAgent(
    name='writer_agent',
    description='An agent that writes feature articles on specific topics.',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
review_agent_remoteA2a = RemoteA2aAgent(
    name='review_agent',
    description='An agent that reviews feature articles.',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

/var/tmp/ipykernel_8184/2951793560.py:19: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent1_remoteA2a = RemoteA2aAgent(
/var/tmp/ipykernel_8184/2951793560.py:31: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent2_remoteA2a = RemoteA2aAgent(
/var/tmp/ipykernel_8184/2951793560.py:43: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implemen

In [18]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
        
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## The research agent will create a research report.\n---\n'),
        get_print_agent('\n## Selecting the topics for investigation.\n'),
        research_agent1_remoteA2a,
        get_print_agent('\n## Creating the research report based on the selected topics.\n'),
        research_agent2_remoteA2a,
        get_print_agent('\n#### The research report is ready. Shall we proceed with writing the article?\n'),
    ],
    description='An agent that collects the information necessary for writing an article and compiles it into a report.',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## The writer agent will now write the article.\n---\n'),
        writer_agent_remoteA2a,
        get_print_agent('\n---\n## The review agent will now review the article.\n---\n'),
        review_agent_remoteA2a,
        get_print_agent('\n#### Would you like to request a revision of the article based on the review?\n'),
    ],
    description='Create and review an article.',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
If asked what you can do, please respond with a clear and friendly summary of the following process:

- Execute a workflow to create an article on a theme specified by the user.
- First, create a research report on the theme.
- After that, the writer and review agents will collaborate to create an article that adheres to the editorial policy.

If the user specifies a theme for the article, execute the following flow:

1. Announce that you will begin creating an article on that theme and forward the task to research_agent to request a research report.
2. If the user confirms the creation of the article, forward the task to write_and_review_agent to request the article's creation and review.
3. If the user wishes to revise the article, forward the task to write_and_review_agent.

**Conditions**
The nickname for research_agent is "the Research Agent".
The nickname for write_and_review_agent is "the Writer and Review Agents".
''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='An agent that executes the workflow for creating an article.'
)

## Executon example

In [19]:
client = LocalApp(root_agent)

query = '''
Hello, what can you do?
'''
result = await client.stream(query)

I can execute a workflow to create an article on a theme specified by you. First, I'll ask the Research Agent to create a research report on the theme. After that, the Writer and Review Agents will collaborate to create an article that adheres to the editorial policy.



In [20]:
query = '''
Create an article with the theme "Spicy Arrangements of Local Cuisine
to Get Through the Summer."
'''
result = await client.stream(query)

Okay, I will begin creating an article on the theme "Spicy Arrangements of Local Cuisine to Get Through the Summer." I'll start by requesting a research report from the Research Agent.


---
## The research agent will create a research report.
---


## Selecting the topics for investigation.



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.

Here are about 5 topics for the research report on "Spicy Arrangements of Local Cuisine to Get Through the Summer":

1.  **The Science and Cultural Rationale: Why Spicy Food Cools You Down in Summer.**
    *   Focus: Explore the physiological effects of capsaicin (inducing sweating, perceived cooling) and the historical/cultural traditions in various hot regions that embrace spicy dishes during warmer months.
2.  **Global Culinary Hotspots: Iconic Local Dishes and Their Summer Spicy Variations.**
    *   Focus: Identify specific regions (e.g., Southeast Asia, Latin America, South Korea, Sichuan China) and present examples of their traditional local dishes that are either inherently spicy or are given specific spicy "arrangements" to be enjoyed during summer.
3.  **Key Ingredients and Flavor Profiles: Crafting the Summer Spicy Experience.**
    *   Focus: Investigate the common chili varieties (e.g., bird's eye, serrano, gochugaru) and other pungent ingredients (ginger, garlic, pepperco

In [21]:
query = '''
Yes, go ahead.
'''
result = await client.stream(query)

Great. I'll now forward the task to the Writer and Review Agents to create and review the article based on the research report.


---
## The writer agent will now write the article.
---



Feeling the heat this summer? While your first instinct might be to reach for an ice-cold drink, what if we told you the secret to staying cool actually involves turning up the heat on your plate? Get ready to explore the wonderfully counter-intuitive world of spicy local cuisines designed to beat the summer swelter!

## Overview

It sounds a bit crazy, right? Eating spicy food when it's already scorching outside. But there's a fascinating science behind why this age-old tradition, practiced in many hot regions worldwide, actually works. When you bite into something spicy, the capsaicin in chilies activates receptors in your mouth, tricking your brain into thinking you're overheating. What happens next is your body's natural cooling system kicking in: you start to sweat! As that sweat evaporates from your skin, it carries heat away, leaving you feeling surprisingly cooler. This isn't just a quirky local custom; it's a physiological hack embraced by cultures from Thailand with their vib

In [22]:
query = '''
Yes, go ahead.
'''
result = await client.stream(query)

Okay, I will forward the article and the review comments to the Writer and Review Agents for revision.


---
## The writer agent will now write the article.
---



# Beat Heat: Eat Spicy Now!

Ever wonder if fighting fire with fire actually works? When the summer heat hits, reaching for that fiery curry or a chili-laden taco might seem counter-intuitive, but science (and tradition!) says it could be your secret weapon against the sweltering temperatures.

## Overview 🌶️
It sounds crazy, but many cultures in hot climates embrace spicy dishes, and there's a good reason why. The active compound in chili peppers, capsaicin, doesn't actually raise your body temperature; instead, it tricks your brain into *thinking* it's hot. This sensation triggers your body's natural cooling mechanism: sweating. As sweat evaporates from your skin, it carries heat away, effectively cooling you down. So, while you might feel a momentary burn, the long-term effect is a refreshing drop in perceived temperature. It's a clever biological hack!

## Latest Information ✨
Recent studies continue to affirm the physiological benefits of capsaicin beyond just cooling. Research pu

In [23]:
query = f'''
Good job, show the entire article again.
'''
result = await client.stream(query)

# Beat Heat: Eat Spicy Now!

Ever wonder if fighting fire with fire actually works? When the summer heat hits, reaching for that fiery curry or a chili-laden taco might seem counter-intuitive, but science (and tradition!) says it could be your secret weapon against the sweltering temperatures.

## Overview 🌶️
It sounds crazy, but many cultures in hot climates embrace spicy dishes, and there's a good reason why. The active compound in chili peppers, capsaicin, doesn't actually raise your body temperature; instead, it tricks your brain into *thinking* it's hot. This sensation triggers your body's natural cooling mechanism: sweating. As sweat evaporates from your skin, it carries heat away, effectively cooling you down. So, while you might feel a momentary burn, the long-term effect is a refreshing drop in perceived temperature. It's a clever biological hack!

## Latest Information ✨
Recent studies continue to affirm the physiological benefits of capsaicin beyond just cooling. Research pu